This quickstart guide explains how to join two tables A and B using edit distance measure. First, you need to import the required packages as follows (if you have installed **py_stringsimjoin** it will automatically install the dependencies **py_stringmatching** and **pandas**):

In [41]:
# Import libraries
import py_stringsimjoin as ssj
import py_stringmatching as sm
import pandas as pd
import os, sys

In [42]:
print('python version: ' + sys.version)
print('py_stringsimjoin version: ' + ssj.__version__)
print('py_stringmatching version: ' + sm.__version__)
print('pandas version: ' + pd.__version__)

python version: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
py_stringsimjoin version: 0.2.0
py_stringmatching version: 0.4.0
pandas version: 0.22.0


Joining two tables using edit distance measure typically consists of three steps:
1. Loading the input tables
2. Profiling the tables
3. Performing the join

# 1. Loading the input tables

We begin by loading the two tables. For the purpose of this guide, 
we use the sample dataset that comes with the package. 

In [46]:
# construct the path of the tables to be loaded. Since we are loading a 
# dataset from the package, we need to access the data from the path 
# where the package is installed. If you need to load your own data, you can directly
# provide your table path to the read_csv command.

table_A_path = os.sep.join([ssj.get_install_path(), 'datasets', 'data', 'imdb_A.csv'])
table_B_path = os.sep.join([ssj.get_install_path(), 'datasets', 'data', 'imdb_B.csv'])

In [48]:
# Load csv files as dataframes.
A = pd.read_csv(table_A_path,error_bad_lines=False)
B = pd.read_csv(table_B_path,error_bad_lines=False)
print('Number of records in A: ' + str(len(A)))
print('Number of records in B: ' + str(len(B)))

Number of records in A: 9901
Number of records in B: 9818


b'Skipping line 137: expected 6 fields, saw 7\nSkipping line 356: expected 6 fields, saw 7\nSkipping line 376: expected 6 fields, saw 7\nSkipping line 423: expected 6 fields, saw 7\nSkipping line 527: expected 6 fields, saw 7\nSkipping line 528: expected 6 fields, saw 8\nSkipping line 557: expected 6 fields, saw 7\nSkipping line 565: expected 6 fields, saw 7\nSkipping line 592: expected 6 fields, saw 7\nSkipping line 733: expected 6 fields, saw 7\nSkipping line 918: expected 6 fields, saw 7\nSkipping line 922: expected 6 fields, saw 7\nSkipping line 979: expected 6 fields, saw 7\nSkipping line 1152: expected 6 fields, saw 9\nSkipping line 1263: expected 6 fields, saw 7\nSkipping line 1337: expected 6 fields, saw 7\nSkipping line 1471: expected 6 fields, saw 7\nSkipping line 1485: expected 6 fields, saw 7\nSkipping line 1500: expected 6 fields, saw 7\nSkipping line 1646: expected 6 fields, saw 7\nSkipping line 1652: expected 6 fields, saw 7\nSkipping line 2038: expected 6 fields, saw 7\

In [49]:
A

,ID,name,year,director,writers,actors
0,1,The Martian,2015,Ridley Scott,Drew Goddard; Andy Weir,Matt Damon; Jessica Chastain; Kristen Wiig
1,2,The Green Inferno,2013,Eli Roth,Guillermo Amoedo; Eli Roth,Lorenza Izzo; Ariel Levy; Aaron Burns
2,3,Everest,2015,Baltasar Kormákur,William Nicholson; Simon Beaufoy,Jason Clarke; Ang Phula Sherpa; Thomas M. Wright
3,4,Sicario,2015,Denis Villeneuve,Taylor Sheridan,Emily Blunt; Josh Brolin; Benicio Del Toro
4,5,The Intern(I),2015,Nancy Meyers,Nancy Meyers,Robert De Niro; Anne Hathaway; Rene Russo
5,6,Black Mass(I),2015,Scott Cooper,Mark Mallouk; Jez Butterworth,Johnny Depp; Benedict Cumberbatch; Dakota Johnson
6,7,Hotel Transylvania 2,2015,Genndy Tartakovsky,Robert Smigel; Adam Sandler,Adam Sandler; Andy Samberg; Selena Gomez
7,8,Maze Runner: The Scorch Trials,2015,Wes Ball,T.S. Nowlin; James Dashner,Dylan O'Brien; Kaya Scodelario; Thomas Brodie-Sangster
8,9,The Visit(I),2015,M. Night Shyamalan,M. Night Shyamalan,Olivia DeJonge; Ed Oxenbould; Deanna Dunagan
9,10,Spectre,2015,Sam Mendes,John Logan; Neal Purvis,Daniel Craig; Christoph Waltz; Ralph Fiennes


In [50]:
B

,ID,title,year,director(s),writer(s),actor(s)
0,0,Jurassic World,2015.0,Colin Trevorrow,Rick Jaffa;Amanda Silver;Derek Connolly;Colin Trevorrow,Chris Pratt;Bryce Dallas Howard;Jake Johnson;Judy Greer
1,1,The Martian,2015.0,Ridley Scott,Drew Goddard;Andy Weir,Matt Damon;Jessica Chastain;Kate Mara;Jeff Daniels
2,2,Terminator Genisys,2015.0,Alan Taylor,Laeta Kalogridis;Patrick Lussier,Arnold Schwarzenegger;Jason Clarke;Emilia Clarke;Jai Courtney
3,3,Mad Max: Fury Road,2015.0,George Miller,George Miller;Nick Lathouris;Brendan McCarthy,Tom Hardy;Charlize Theron;Hugh Keays-Byrne;Nicholas Hoult
4,4,San Andreas,2015.0,Brad Peyton,Carlton Cuse;Andre Fabrizio;Jeremy Passmore,Alexandra Daddario;Dwayne Johnson;Carla Gugino;Archie Panjabi
5,5,Southpaw,2015.0,Antoine Fuqua,Kurt Sutter;Richard Wenk,Jake Gyllenhaal;Rachel McAdams;Forest Whitaker;Oona Laurence
6,6,Tomorrowland,2015.0,Brad Bird,Brad Bird;Damon Lindelof;Jeff Jensen,Britt Robertson;George Clooney;Raffey Cassidy;Thomas Robinson
7,7,Avengers: Age of Ultron,2015.0,Joss Whedon,Joss Whedon,Robert Downey Jr.;Chris Evans;Chris Hemsworth;Mark Ruffalo
8,8,Pixels,2015.0,Chris Columbus,Tim Herlihy;Adam Sandler;Timothy Dowling;Patrick Jean,Adam Sandler;Michelle Monaghan;Peter Dinklage;Josh Gad
9,9,Minions,2015.0,Kyle Balda;Pierre Coffin,Brian Lynch,Pierre Coffin;Sandra Bullock;Jon Hamm;Katy Mixon


# 2. Profiling the tables

Before performing the join, we may want to profile the tables to 
know about the characteristics of the attributes. This can help identify:

a) unique attributes in the table which can be used as key attribute when performing 
   the join. A key attribute is needed to uniquely identify a tuple. 
   
b) the number of missing values present in each attribute. This can 
   help you in deciding the attribute on which to perform the join. 
   For example, an attribute with a lot of missing values may not be a good 
   join attribute. Further, based on the missing value information you 
   need to decide on how to handle missing values when performing the join 
   (See the section below on 'Handling missing values' to know more about
   the options available for handling missing values when performing the join).
   
You can profile the attributes in a table using the following command:

In [37]:
# profile attributes in table A
ssj.profile_table_for_join(A)

,Unique values,Missing values,Comments
Attribute,,,
A.id,7 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
A.name,7 (100.0%),1 (14.29%),Joining on this attribute will ignore 1 (14.29%) rows.
A.birth_year,6 (85.71%),0 (0.0%),
A.hourly_wage,7 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
A.address,7 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
A.zipcode,3 (42.86%),1 (14.29%),Joining on this attribute will ignore 1 (14.29%) rows.


In [38]:
# profile attributes in table B
ssj.profile_table_for_join(B)

,Unique values,Missing values,Comments
Attribute,,,
B.id,8 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
B.name,8 (100.0%),1 (12.5%),Joining on this attribute will ignore 1 (12.5%) rows.
B.birth_year,6 (75.0%),0 (0.0%),
B.hourly_wage,8 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
B.address,8 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
B.zipcode,3 (37.5%),1 (12.5%),Joining on this attribute will ignore 1 (12.5%) rows.


If the input tables does not contain any key attribute, then you need 
to create a key attribute. In the current example, both the input tables
A and B have key attributes, and hence you can proceed to the next step.
In the case the table does not have any key attribute, you can 
add a key attribute using the following command:

In [39]:
B['new_key_attr'] = range(0, len(B))
B

,B.id,B.name,B.birth_year,B.hourly_wage,B.address,B.zipcode,new_key_attr
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107.0,0
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107.0,1
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122.0,2
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122.0,3
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122.0,4
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107.0,5
6,b7,,1990,40.0,"24th Street, San Francisco",94122.0,6
7,b8,NaN,1986,25.0,"20th Street, San Francisco",NaN,7


For the purpose of this guide, we will now join tables A and B on 
'name' attribute using edit distance measure. Next, we need to decide on what 
threshold to use for the join. For this guide, we will use a threshold of 5. 
Specifically, the join will now find tuple pairs from A and B such that 
the edit distance over the 'name' attributes is at most 5.

# 3. Performing the join

The next step is to perform the edit distance join using the following command:

In [40]:
# find all pairs from A and B such that the edit distance
# on 'name' is at most 5.
# l_out_attrs and r_out_attrs denote the attributes from the 
# left table (A) and right table (B) that need to be included in the output.

output_pairs = ssj.edit_distance_join_disk_cy(A, B, 'A.id', 'B.id', 'A.name', 'B.name', 5,2, 
                                      l_out_attrs=['A.name'], r_out_attrs=['B.name'])

AttributeError: module 'py_stringsimjoin' has no attribute 'edit_distance_join_disk_cy'

In [11]:
len(output_pairs)

4

In [12]:
# examine the output pairs
output_pairs

,_id,l_A.id,r_B.id,l_A.name,r_B.name,_sim_score
0,0,a3,b2,William Bridge,Bill Bridge,4.0
1,1,a2,b3,Michael Franklin,Mike Franklin,4.0
2,2,a5,b5,Alphonse Kemper,Alfons Kemper,3.0
3,3,a6,b7,,,0.0


# Handling missing values

By default, pairs with missing values are not included
in the output. This is because a string with a missing value
can potentially match with all strings in the other table and 
hence the number of output pairs can become huge. If you want 
to include pairs with missing value in the output, you need to 
set the **allow_missing** flag to True, as shown below:

In [13]:
output_pairs = ssj.edit_distance_join(A, B, 'A.id', 'B.id', 'A.name', 'B.name', 5, allow_missing=True, 
                                      l_out_attrs=['A.name'], r_out_attrs=['B.name'])

0%   100%
[#######] | ETA: 00:00:00
Total time elapsed: 00:00:00
0%  100%
[##] | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:00


In [14]:
output_pairs

,_id,l_A.id,r_B.id,l_A.name,r_B.name,_sim_score
0,0,a3,b2,William Bridge,Bill Bridge,4.0
1,1,a2,b3,Michael Franklin,Mike Franklin,4.0
2,2,a5,b5,Alphonse Kemper,Alfons Kemper,3.0
3,3,a6,b7,,,0.0
0,4,a7,b1,NaN,Mark Levene,NaN
1,5,a7,b2,NaN,Bill Bridge,NaN
2,6,a7,b3,NaN,Mike Franklin,NaN
3,7,a7,b4,NaN,Joseph Kuan,NaN
4,8,a7,b5,NaN,Alfons Kemper,NaN
5,9,a7,b6,NaN,Michael Brodie,NaN


# Enabling parallel processing

If you have multiple cores which you want to exploit for performing the 
join, you need to use the **n_jobs** option. If n_jobs is -1, all CPUs 
are used. If 1 is given, no parallel computing code is used at all, 
which is useful for debugging and is the default option. For n_jobs below 
-1, (n_cpus + 1 + n_jobs) are used (where n_cpus is the total number of 
CPUs in the machine). Thus for n_jobs = -2, all CPUs but one are used. If 
(n_cpus + 1 + n_jobs) becomes less than 1, then no parallel computing code 
will be used (i.e., equivalent to the default).

The following command exploits all the cores available to perform the join:

In [15]:
output_pairs = ssj.edit_distance_join(A, B, 'A.id', 'B.id', 'A.name', 'B.name', 5, 
                                      l_out_attrs=['A.name'], r_out_attrs=['B.name'], n_jobs=-1)

0%   100%
[#######] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [16]:
len(output_pairs)

4

You need to set n_jobs to 1 when you are debugging or you do not want 
to use any parallel computing code. If you want to execute the join as 
fast as possible, you need to set n_jobs to -1 which will exploit all 
the CPUs in your machine. In case there are other concurrent processes 
running in your machine and you do not want to halt them, then you may 
need to set n_jobs to a value below -1.

# Performing join on numeric attributes

The join method expects the join attributes to be of string type. 
If you need to perform the join over numeric attributes, then you need 
to first convert the attributes to string type and then perform the join.
For example, if you need to join 'A.zipcode' in table A with 'B.zipcode' in
table B, you need to first convert the attributes to string type using 
the following command:

In [17]:
ssj.dataframe_column_to_str(A, 'A.zipcode', inplace=True)
ssj.dataframe_column_to_str(B, 'B.zipcode', inplace=True)

/home/bhargav/Downloads/stringjoin/disk_feature/py_stringsimjoin/utils/converter.py:115: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif pd.np.issubdtype(col_type, pd.np.float):
/home/bhargav/Downloads/stringjoin/disk_feature/py_stringsimjoin/utils/converter.py:115: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif pd.np.issubdtype(col_type, pd.np.float):


True

Note that the above command preserves the NaN values while converting the numeric column to string type. Next, you can perform the join as shown below:

In [18]:
output_pairs = ssj.edit_distance_join(A, B, 'A.id', 'B.id', 'A.zipcode', 'B.zipcode', 1, 
                                      l_out_attrs=['A.zipcode'], r_out_attrs=['B.zipcode'])
output_pairs

0%   100%
[#######] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,l_A.id,r_B.id,l_A.zipcode,r_B.zipcode,_sim_score
0,0,a1,b1,94107,94107,0.0
1,1,a3,b1,94107,94107,0.0
2,2,a1,b2,94107,94107,0.0
3,3,a3,b2,94107,94107,0.0
4,4,a2,b3,94122,94122,0.0
5,5,a4,b3,94122,94122,0.0
6,6,a5,b3,94122,94122,0.0
7,7,a6,b3,94122,94122,0.0
8,8,a2,b4,94122,94122,0.0
9,9,a4,b4,94122,94122,0.0


# Additional options

You can find all the options available for the edit distance 
join function using the **help** command as shown below:

In [19]:
help(ssj.edit_distance_join)

Help on function edit_distance_join in module py_stringsimjoin.join.edit_distance_join:

edit_distance_join(ltable, rtable, l_key_attr, r_key_attr, l_join_attr, r_join_attr, threshold, comp_op='<=', allow_missing=False, l_out_attrs=None, r_out_attrs=None, l_out_prefix='l_', r_out_prefix='r_', out_sim_score=True, n_jobs=1, show_progress=True, tokenizer=<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7faef034ab00>)



# More information

Similar to edit distance measure, you can use the package to perform 
join using other measures such as cosine, Dice, Jaccard, overlap and 
overlap coefficient. For measures such as TF-IDF which are not 
directly supported, you can perform the join using the filters provided 
in the package. To know more about other join methods as well as how to 
use filters, refer to the how-to guide (available from the 
[package homepage](https://sites.google.com/site/anhaidgroup/projects/py_stringsimjoin)).